In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import time
from datetime import datetime, timedelta

all_currency_pairs = ['USDCHF10080',
                 'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]



headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]


In [2]:
raw_df = pd.read_csv("../raw_weekly_data/NZDCAD10080.csv", names = headers )

needed_weeks_df = raw_df.loc[(raw_df['date'] >= "2015.01.01")   ]   

needed_weeks_df = needed_weeks_df.reset_index(drop=True)

available_weeks = needed_weeks_df['date'].values



#print(raw_df)
print(available_weeks)

['2015.01.04' '2015.01.11' '2015.01.18' '2015.01.25' '2015.02.01'
 '2015.02.08' '2015.02.15' '2015.02.22' '2015.03.01' '2015.03.08'
 '2015.03.15' '2015.03.22' '2015.03.29' '2015.04.05' '2015.04.12'
 '2015.04.19' '2015.04.26' '2015.05.03' '2015.05.10' '2015.05.17'
 '2015.05.24' '2015.05.31' '2015.06.07' '2015.06.14' '2015.06.21'
 '2015.06.28' '2015.07.05' '2015.07.12' '2015.07.19' '2015.07.26'
 '2015.08.02' '2015.08.09' '2015.08.16' '2015.08.23' '2015.08.30'
 '2015.09.06' '2015.09.13' '2015.09.20' '2015.09.27' '2015.10.04'
 '2015.10.11' '2015.10.18' '2015.10.25' '2015.11.01' '2015.11.08'
 '2015.11.15' '2015.11.22' '2015.11.29' '2015.12.06' '2015.12.13'
 '2015.12.20' '2015.12.27' '2016.01.03' '2016.01.10' '2016.01.17'
 '2016.01.24' '2016.01.31' '2016.02.07' '2016.02.14' '2016.02.21'
 '2016.02.28' '2016.03.06' '2016.03.13' '2016.03.20' '2016.03.27'
 '2016.04.03' '2016.04.10' '2016.04.17' '2016.04.24' '2016.05.01'
 '2016.05.08' '2016.05.15' '2016.05.22' '2016.05.29' '2016.06.05'
 '2016.06.

In [3]:
for y in available_weeks:
    
    current_date = datetime.strptime(y, '%Y.%m.%d').strftime('%A')
    
    if current_date != 'Sunday':
        available_weeks = [x for x in available_weeks if x != y]
        print(y)

In [4]:
for a in available_weeks:
  
  
    path = "./files/" + a

    try: 
        os.makedirs(path)
        control_file = pd.DataFrame()
        
        for  b in range(8):
            control_file['control' + str(b)] = -1
            
            
        control_file.to_csv(path + '/control.csv')
        
        

    except OSError as error: 
         print(error)  
 

In [5]:
all_currency_pair_df = []

for d in all_currency_pairs:
    
    current_currency_df = pd.read_csv("../raw_weekly_data/" + d + ".csv", names = headers ) 
    
    current_currency_df = current_currency_df.iloc[::-1]
    current_currency_df = current_currency_df.reset_index(drop=True)
      
    current_currency_df = {'name': d.replace('10080',''),'dataframe': current_currency_df }
    
    all_currency_pair_df.append(current_currency_df)
    

print(all_currency_pair_df)

    
    

[{'name': 'USDCHF', 'dataframe':             date ignore     open     high      low    close  volume
0     2021.12.05  00:00  0.91625  0.92690  0.91625  0.92396   37199
1     2021.11.28  00:00  0.92287  0.92729  0.91574  0.91767  170181
2     2021.11.21  00:00  0.92768  0.93736  0.92151  0.92393  154242
3     2021.11.14  00:00  0.91932  0.93297  0.91869  0.92866  153577
4     2021.11.07  00:00  0.91268  0.92377  0.91016  0.92097  150396
...          ...    ...      ...      ...      ...      ...     ...
2597  1971.01.31  00:00  4.29680  4.29690  4.29650  4.29690       5
2598  1971.01.24  00:00  4.30050  4.30050  4.29630  4.29630       5
2599  1971.01.17  00:00  4.30290  4.30290  4.29960  4.29960       5
2600  1971.01.10  00:00  4.31020  4.31020  4.30760  4.30760       5
2601  1971.01.03  00:00  4.31800  4.31800  4.31030  4.31090       5

[2602 rows x 7 columns]}, {'name': 'GBPUSD', 'dataframe':             date ignore     open     high      low    close  volume
0     2021.12.05  00:00 

In [6]:
acceptable_signal_percent = 70

analysis_span = 52

analysis_interval = 3

no_of_weeks_analysis = [c for c in range(0, analysis_span, analysis_interval)]

all_time_next_week_trade =[]

for e in available_weeks:
    
    next_week_trade = []

    for f in all_currency_pair_df:
        
        
        current_currency_df = f['dataframe'].copy(deep = True)

        current_currency_df = current_currency_df.loc[(current_currency_df['date'] <= e )   ] 
        
        
        
        current_currency_df = current_currency_df.reset_index(drop=True)
        
        

        current_currency_df = current_currency_df.head(52)
        
        #print(f['name'], current_currency_df)

        needed_value_list = []
        
        needed_direction_list = []
        
        for g in no_of_weeks_analysis:
            
            needed_value = current_currency_df.copy()['close'][g]
            
            needed_value_list.append(needed_value)
            
            
        
        #print(f['name'], needed_value_list)
       
        
        for index, h in enumerate(needed_value_list):
            
            if index == 0:
                
                continue
                
            if h - needed_value_list[index-1] > 0 :
                
                needed_direction_list.append('S')
            
            else:
                
                needed_direction_list.append('B')
                
                
        #print(f['name'],needed_direction_list)  
        
        if (100 * needed_direction_list.count('S') / len(needed_direction_list) ) > acceptable_signal_percent:
            
            next_week_trade.append({'name': f['name'], 'direction': 'S'})
        
        if ( 100 * needed_direction_list.count('B') / len(needed_direction_list) ) > acceptable_signal_percent:
            
            next_week_trade.append({'name': f['name'], 'direction': 'B'})
    
    
    
    if len(next_week_trade) > 0:
        
        all_time_next_week_trade.append( {'week': e, 'next_week_trades': next_week_trade} )
    
    
    
    
    #print('--------------------------------------------------------------------------------------------', e )
    #print(next_week_trade)
    #print(len(next_week_trade))
    
    
    
if len(all_time_next_week_trade) == 0:
             
    raise Exception("no week passed the acceptable_signal_percent ")
        
        
    
print(len(all_time_next_week_trade))

316


In [7]:
# creating a dataframe for nextweek trades and saving it in csv

for i in all_time_next_week_trade:
    
    current_week_data = i['next_week_trades']
    current_week_name = i['week']
    
    currency_name =[]
    trade_direction = []
    
    for j in current_week_data:
        
        currency_name.append(j['name'])
        trade_direction.append(j['direction'])
        
        
    
    
    
    
    next_week_trades_dataframe = pd.DataFrame()
    next_week_trades_dataframe['name']= currency_name
    next_week_trades_dataframe['direction'] = trade_direction
    
    
    print(next_week_trades_dataframe)

    next_week_trades_dataframe.to_csv("files/" + current_week_name + '/next_week_trades.csv')

     name direction
0  GBPUSD         S
1  EURUSD         S
2  EURCHF         S
     name direction
0  EURUSD         S
1  EURCHF         S
2  EURGBP         S
3  EURCAD         S
     name direction
0  EURUSD         S
1  EURCHF         S
2  EURGBP         S
     name direction
0  GBPUSD         S
1  EURUSD         S
2  EURGBP         S
     name direction
0  EURUSD         S
1  EURCHF         S
2  EURGBP         S
     name direction
0  USDCHF         B
1  EURUSD         S
2  GBPCHF         B
3  EURGBP         S
     name direction
0  GBPUSD         S
1  EURUSD         S
2  GBPCHF         B
3  EURGBP         S
4  EURNZD         S
     name direction
0  USDCHF         B
1  EURUSD         S
2  GBPCHF         B
3  EURGBP         S
4  EURJPY         S
5  EURCAD         S
     name direction
0  USDCHF         B
1  EURUSD         S
2  USDCAD         B
3  AUDUSD         S
4  NZDUSD         S
5  GBPCHF         B
6  EURGBP         S
     name direction
0  GBPUSD         S
1  EURUSD         S


     name direction
0  EURCHF         B
     name direction
0  EURUSD         B
1  NZDCAD         S
     name direction
0  EURCHF         B
     name direction
0  EURUSD         B
1  NZDCAD         S
     name direction
0  EURCHF         B
     name direction
0  EURUSD         B
1  NZDCAD         S
     name direction
0  EURCHF         B
1  EURJPY         B
2  EURAUD         B
     name direction
0  EURUSD         B
     name direction
0  EURUSD         B
     name direction
0  EURCHF         B
     name direction
0  EURUSD         B
     name direction
0  EURCHF         B
1  AUDNZD         S
     name direction
0  EURUSD         B
1  EURCAD         B
2  EURAUD         B
     name direction
0  EURUSD         B
1  EURCHF         B
2  EURNZD         B
     name direction
0  EURCHF         B
1  EURJPY         B
2  EURAUD         B
3  AUDNZD         S
     name direction
0  EURUSD         B
1  EURCHF         B
     name direction
0  EURCHF         B
1  EURNZD         B
2  AUDCAD         S


     name direction
0  GBPUSD         B
1  AUDUSD         B
2  NZDUSD         B
3  CHFJPY         B
4  GBPJPY         B
5  EURJPY         B
6  EURNZD         S
7  CADJPY         B
8  AUDJPY         B
9  NZDJPY         B
     name direction
0  USDCAD         S
1  GBPCHF         B
2  EURGBP         S
3  GBPJPY         B
4  EURJPY         B
5  EURNZD         S
6  CADJPY         B
7  AUDJPY         B
     name direction
0  GBPUSD         B
1  USDCAD         S
2  CHFJPY         B
3  EURGBP         S
4  GBPJPY         B
5  EURJPY         B
6  CADJPY         B
     name direction
0  GBPUSD         B
1  CHFJPY         B
2  GBPJPY         B
3  EURJPY         B
4  EURCAD         S
5  EURNZD         S
6  CADJPY         B
7  AUDJPY         B
     name direction
0  USDCAD         S
1  EURGBP         S
2  GBPJPY         B
3  EURJPY         B
4  EURNZD         S
5  CADJPY         B
6  AUDJPY         B
     name direction
0  GBPUSD         B
1  USDCAD         S
2  EURGBP         S
3  GBPJPY         B


In [8]:
all_currency_thirty_minutes_data = {}

for z in all_currency_pairs:
    current_currency = z.replace('10080','')
    current_currency_df = pd.read_csv('../raw_30_minutes_data/' + current_currency + '30.csv', names=headers ) 
    
    all_currency_thirty_minutes_data[current_currency] = current_currency_df
    
    #print( current_currency, )
    
    
print(len(all_currency_thirty_minutes_data))




27


In [9]:
qualified_weeks =[]

for k in all_time_next_week_trade:
    
    current_week_data = k['next_week_trades']
    current_week_date = k['week']
    
    current_week_date = datetime.strptime(current_week_date, '%Y.%m.%d')

    week_start_date  = current_week_date + timedelta(days=7)
    week_end_date = current_week_date + timedelta(days=12)
    
    current_week_date = current_week_date.strftime("%Y.%m.%d")
    week_start_date = week_start_date.strftime("%Y.%m.%d")
    week_end_date = week_end_date.strftime("%Y.%m.%d")      

    
    

    #print(current_week_date, week_start_date, week_end_date)
    
    qualified_currencies = 0
   
    for l in current_week_data:
        
        
        current_currency = l['name'] 
        
        current_currency_df = all_currency_thirty_minutes_data[l['name']].copy()
        
        
        
        
        current_currency_df = current_currency_df.loc[  (current_currency_df['date'] >= week_start_date)
                                                   
                                                                                 & 
                                                              (current_currency_df['date'] <= week_end_date)
                                                            ]  
    
        
        
        
        current_currency_df = current_currency_df.reset_index(drop=True )
        #print( week_start_date, current_currency, current_currency_df)
        
       
        current_currency_dataframe_row_ammount = len(current_currency_df.index)
        
        if  current_currency_dataframe_row_ammount < 238:
            
            print(current_currency,len(current_currency_df),week_start_date,'less than 238, skipping ',current_week_date)
            continue
            #raise Exception("lenght is lesss than 238")
            
            
        
        if  current_currency_dataframe_row_ammount != 240:
            
            if (current_currency_df['ignore'].tail(1).values == '22:30' ):
                
                
                
                last_row = current_currency_df.copy().tail(1)
                
                last_row.tail()['ignore'] = '23:00'
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
                
                last_row.tail()['ignore'] = '23:30'
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
                #print(current_currency, current_currency_df)
                
                if  len(current_currency_df.index) != 240:
              
                    print(current_currency, current_currency_df)
                    raise Exception("number of rows is still wrong after repair")
        
            else:
                
                print(current_currency,len(current_currency_df),week_start_date,"irrepairable,skipping ",current_week_date)
                continue
        
        if current_currency_df['ignore'].iloc[-1] != '23:30' :
        
            print(current_currency, current_currency_df)
            raise Exception("last row is not 23:30")
        
        
      
            
             
        
        
       
        current_currency_df.to_csv("files/" + k['week'] + '/' + current_currency +".csv", index=None)
        
        qualified_currencies = qualified_currencies +1
        
        
        
    if qualified_currencies == len(current_week_data):
            
        qualified_weeks.append(k)
        
    
   
        
        #print(current_currency_df)
        
        
        
    #print('-----------------------------------------------------------------------------------------------', week_start_date)


print(len(qualified_weeks))


EURCHF 0 2015.01.11 less than 238, skipping  2015.01.04
EURCHF 239 2015.02.08 irrepairable,skipping  2015.02.01
USDCAD 182 2015.12.20 less than 238, skipping  2015.12.13
GBPCAD 182 2015.12.20 less than 238, skipping  2015.12.13
NZDJPY 182 2015.12.20 less than 238, skipping  2015.12.13
USDCAD 184 2015.12.27 less than 238, skipping  2015.12.20
GBPUSD 237 2016.10.09 less than 238, skipping  2016.10.02
EURGBP 237 2016.10.09 less than 238, skipping  2016.10.02
GBPJPY 237 2016.10.09 less than 238, skipping  2016.10.02
GBPCAD 228 2016.12.25 less than 238, skipping  2016.12.18
NZDCAD 218 2017.09.03 less than 238, skipping  2017.08.27
GBPJPY 239 2017.09.10 irrepairable,skipping  2017.09.03
GBPCAD 239 2017.09.10 irrepairable,skipping  2017.09.03
EURCHF 180 2017.12.24 less than 238, skipping  2017.12.17
USDCAD 174 2018.12.23 less than 238, skipping  2018.12.16
CADJPY 174 2018.12.23 less than 238, skipping  2018.12.16
GBPUSD 178 2018.12.30 less than 238, skipping  2018.12.23
USDCAD 178 2018.12.30 

In [10]:
timeframe = []
trade_values = []
unique_time_frames = []

for m in qualified_weeks:

    
    
    current_week = m['week']
    current_week_data = m['next_week_trades']
    
    current_week_drawdown_df = pd.DataFrame()
    
    current_week_profit = pd.DataFrame()
  

    
    
    for index, n in enumerate(current_week_data):
        
        
        
        current_currency = n['name']
        current_currency_direction = n['direction']
        current_currency_df =  pd.read_csv("files/" + current_week + '/' + current_currency +".csv" ) 
        
        
        # adding timeframe to the maximal draw down df
        if index == 0:
            
            current_week_drawdown_df['date'] = current_currency_df['date']
            current_week_profit['date'] = current_currency_df['date']
           
            #saving unique_timeframes in a week to use later in the script
            if len(unique_time_frames) == 0:
                
                unique_time_frames = current_currency_df['ignore'].values
            
        
        
        #print(current_currency_df)
        maximal_draw_down =[]
        profit = []
        pips_corrector = 100000
            
        if current_currency.endswith('JPY'):
                
            pips_corrector = 1000
            
        
        
        
        for o in range(240):
                   
            if current_currency_direction == 'B':
                
                
                dd = current_currency_df.loc[o,'low'] - current_currency_df.loc[0,'open']
                p = current_currency_df.loc[o,'close'] - current_currency_df.loc[0,'open']
              
            if current_currency_direction == 'S':
                
                
                dd = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'high']
                p = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'close']
            
            
            maximal_draw_down.append(dd*pips_corrector)
            profit.append(p*pips_corrector)
                
        
        current_week_drawdown_df[current_currency] = maximal_draw_down
        current_week_profit[current_currency] = profit
            
    current_week_drawdown_df['empty'] = ['' for p in range(240)]      
    current_week_drawdown_df["sum"] = current_week_drawdown_df.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_drawdown_df.to_csv("files/" + m['week'] + '/' + 'maximal_draw_down.csv', index= None)
    
    
    
    current_week_profit['empty'] = ['' for p in range(240)]      
    current_week_profit["sum"] = current_week_profit.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_profit.to_csv("files/" + m['week'] + '/' + 'profit.csv', index= None)
    
    
    
    #print(current_week_drawdown_df)
        
    
    
    
    
    
    
    
    
    
    print('--------------------------------------------------------------------------------------------------', m)   
    
    
print(current_week_drawdown_df)

print(unique_time_frames)

-------------------------------------------------------------------------------------------------- {'week': '2015.01.11', 'next_week_trades': [{'name': 'EURUSD', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.01.18', 'next_week_trades': [{'name': 'EURUSD', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.01.25', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.02.08', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name'

-------------------------------------------------------------------------------------------------- {'week': '2015.08.09', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'B'}, {'name': 'USDCAD', 'direction': 'B'}, {'name': 'AUDUSD', 'direction': 'S'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'NZDJPY', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.08.16', 'next_week_trades': [{'name': 'USDCAD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'S'}, {'name': 'AUDCHF', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.08.23', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'USDCAD', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '201

-------------------------------------------------------------------------------------------------- {'week': '2016.07.31', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2016.08.07', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'GBPAUD', 'direction': 'S'}, {'name': 'EURJPY', 'direction': 'S'}, {'name': 'EURAUD', 'direction': 'S'}, {'name': 'AUDJPY', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2016.08.14', 'next_week_trades': [{'name': 'AUDUSD', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'GBPJPY', 'direction': 'S'}, {'name': 'EURJPY', 'direction': 'S'}, {'name': 'CADJPY', 'direction': 'S'}]}
------------------------------------

-------------------------------------------------------------------------------------------------- {'week': '2017.07.02', 'next_week_trades': [{'name': 'GBPJPY', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2017.07.09', 'next_week_trades': [{'name': 'CHFJPY', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2017.07.16', 'next_week_trades': [{'name': 'AUDJPY', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2017.07.23', 'next_week_trades': [{'name': 'GBPJPY', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2017.07.30', 'next_week_trades': [{'name': 'CHFJPY', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}]}
----------------------------------------------------

-------------------------------------------------------------------------------------------------- {'week': '2018.07.29', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'S'}, {'name': 'NZDCHF', 'direction': 'S'}, {'name': 'AUDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2018.08.05', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2018.08.12', 'next_week_trades': [{'name': 'AUDUSD', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2018.08.19', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'S'}, {'name': 'NZDCHF', 'direction': 'S'}, {'name'

-------------------------------------------------------------------------------------------------- {'week': '2019.06.09', 'next_week_trades': [{'name': 'AUDUSD', 'direction': 'S'}, {'name': 'NZDUSD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2019.06.16', 'next_week_trades': [{'name': 'GBPCHF', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2019.06.23', 'next_week_trades': [{'name': 'EURJPY', 'direction': 'S'}, {'name': 'NZDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2019.06.30', 'next_week_trades': [{'name': 'USDJPY', 'direction': 'S'}, {'name': 'AUDNZD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2019.07.07', 'next_week_trades': [{'name': 'GBPCH

-------------------------------------------------------------------------------------------------- {'week': '2020.04.26', 'next_week_trades': [{'name': 'EURCHF', 'direction': 'S'}, {'name': 'CADCHF', 'direction': 'S'}, {'name': 'EURJPY', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2020.05.03', 'next_week_trades': [{'name': 'EURCHF', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2020.05.10', 'next_week_trades': [{'name': 'EURCHF', 'direction': 'S'}, {'name': 'NZDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2020.05.17', 'next_week_trades': [{'name': 'EURCHF', 'direction': 'S'}, {'name': 'EURJPY', 'direction': 'S'}]}
----------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------- {'week': '2021.02.21', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'B'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'NZDCHF', 'direction': 'B'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'AUDJPY', 'direction': 'B'}, {'name': 'NZDJPY', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2021.02.28', 'next_week_trades': [{'name': 'USDCAD', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'AUDJPY', 'direction': 'B'}, {'name': 'AUDCAD', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2021.03.07', 'next_week_trades': 

-------------------------------------------------------------------------------------------------- {'week': '2021.07.04', 'next_week_trades': [{'name': 'USDCAD', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'CADJPY', 'direction': 'B'}, {'name': 'AUDJPY', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2021.07.11', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'B'}, {'name': 'USDCAD', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'CADJPY', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2021.07.18', 'next_week_trades': [{'name': 'GBPJPY', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'EURCAD', 'direction': 'S'},

In [11]:
day_time_frames = []

for index, w in enumerate(unique_time_frames):
    
    if index >= 192: 
        day = 'FRI'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >= 144: 
        day = 'THUR'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=96: 
        day = 'WED'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=48: 
        day = 'TUE'  
        day_time_frames.append(day + ' ' + w)
        
    else: 
        day = 'MON'  
        day_time_frames.append(day + ' ' + w)
    
   
    
print(len(day_time_frames))

240


In [12]:
#combining all maximal drawdowns for all weeks
combine_week_drawdown_df = pd.DataFrame()

combine_week_drawdown_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_dd_df = pd.read_csv("files/" + current_week + '/' + 'maximal_draw_down' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_drawdown_df[current_week] = current_week_dd_df['sum']
    

combine_week_drawdown_df['empty'] = ['' for r in range(240)]      
combine_week_drawdown_df["sum"] = combine_week_drawdown_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_drawdown_df.to_csv("files/" + 'combine_week_drawdown_df.csv', index= None)

print(combine_week_drawdown_df)

C:\Users\AKINSH~1\AppData\Local\Temp/ipykernel_10612/567915654.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  combine_week_drawdown_df[current_week] = current_week_dd_df['sum']


          date  2015.01.11  2015.01.18  2015.01.25  2015.02.08  2015.02.15  \
0    MON 00:00      -460.0      -118.0      -213.0       -87.0      -208.0   
1    MON 00:30      -421.0       -53.0      -139.0      -248.0       -86.0   
2    MON 01:00      -794.0        84.0       -77.0      -190.0      -175.0   
3    MON 01:30      -726.0       157.0        27.0      -166.0       -92.0   
4    MON 02:00       -23.0      -904.0        47.0      -373.0       115.0   
..         ...         ...         ...         ...         ...         ...   
235  FRI 21:30      3865.0     -8460.0      -920.0      1746.0      8411.0   
236  FRI 22:00      3983.0     -7716.0     -1053.0      1999.0      8659.0   
237  FRI 22:30      4423.0     -7985.0     -1012.0      2244.0      8715.0   
238  FRI 23:00      4176.0     -7873.0     -1036.0      2069.0      8592.0   
239  FRI 23:30      4485.0     -7828.0     -1086.0      1515.0      8453.0   

     2015.02.22  2015.03.01  2015.03.08  2015.03.15  ...  2021.

C:\Users\AKINSH~1\AppData\Local\Temp/ipykernel_10612/567915654.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  combine_week_drawdown_df['empty'] = ['' for r in range(240)]


In [13]:
#combining all profits drawdowns for all weeks
combine_week_profit_df = pd.DataFrame()

combine_week_profit_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_profit_df = pd.read_csv("files/" + current_week + '/' + 'profit' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_profit_df[current_week] = current_week_profit_df['sum']
    

combine_week_profit_df['empty'] = ['' for r in range(240)]      
combine_week_profit_df["sum"] = combine_week_profit_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_profit_df.to_csv("files/" + 'combine_week_profit_df.csv', index= None)

print(combine_week_profit_df)

C:\Users\AKINSH~1\AppData\Local\Temp/ipykernel_10612/1757970898.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  combine_week_profit_df[current_week] = current_week_profit_df['sum']


          date  2015.01.11  2015.01.18  2015.01.25  2015.02.08  2015.02.15  \
0    MON 00:00      -249.0       143.0       132.0        10.0       162.0   
1    MON 00:30      -293.0       191.0        22.0       -10.0       158.0   
2    MON 01:00      -493.0       681.0       112.0        14.0         8.0   
3    MON 01:30       151.0       180.0       104.0      -123.0       295.0   
4    MON 02:00       196.0      -410.0       102.0      -245.0       556.0   
..         ...         ...         ...         ...         ...         ...   
235  FRI 21:30      4060.0     -7416.0      -847.0      2508.0      8946.0   
236  FRI 22:00      4482.0     -7591.0      -933.0      2901.0      8956.0   
237  FRI 22:30      5126.0     -7711.0      -887.0      2330.0      8893.0   
238  FRI 23:00      4797.0     -7558.0      -934.0      2202.0      8774.0   
239  FRI 23:30      5657.0     -7682.0      -619.0      1802.0      8874.0   

     2015.02.22  2015.03.01  2015.03.08  2015.03.15  ...  2021.

C:\Users\AKINSH~1\AppData\Local\Temp/ipykernel_10612/1757970898.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  combine_week_profit_df['empty'] = ['' for r in range(240)]


In [14]:
#loading all profit and drawdown dataframe to memory to speed up processing

qualified_weeks_dfs=[]

for v in qualified_weeks:
    
    current_week = v['week']

    current_week_profit_df = pd.read_csv("files/" + current_week + '/profit.csv')
    current_week_drawdown_df = pd.read_csv("files/" + current_week + '/maximal_draw_down.csv')
    
    qualified_weeks_dfs.append( 
                                { 'week': current_week,
                                   'profit_df' : current_week_profit_df,
                                   'drawdown_df': current_week_drawdown_df
                                }
                                )
    

print(len(qualified_weeks_dfs))
    
    

301


In [15]:
statistic_df = pd.DataFrame()

start_date_index_list =[]

cut_off_date_index_list =[]

draw_down_list = []

total_spread_list = []

profit_list=[]

profit_ratio_list=[]


combine_week_drawdown_df = pd.read_csv("files/" + 'combine_week_drawdown_df.csv')
combine_week_profit_df = pd.read_csv("files/" + 'combine_week_profit_df.csv')


#print(combine_week_drawdown_df)

start_date_index = [f for f in range(0, 239)]


draw_down = [f*-1 for f in range(100, 3000, 100)] 


total_spread = -200





for t in start_date_index:
    
    cut_off_date_index =   [f+1 for f in range( 1,(240-t)  ) ]
    
    #print(cut_off_date_index)
    
    for g in cut_off_date_index:
        
        
        for q in draw_down :
            
            
            strategy_profit = []
            
            current_maximal_drawdown = q + total_spread
            
            
            for s in qualified_weeks_dfs:
                
                current_week_calculated_profit = None
                
                current_drawdown_df = s['drawdown_df'].copy()
                current_drawdown_df = current_drawdown_df.iloc[t:]
                current_drawdown_df = current_drawdown_df.reset_index(drop=True)   
                current_drawdown_df = current_drawdown_df.iloc[:g]
                current_week_drawdown_list = current_drawdown_df['sum'].values
                
                

                current_profit_df = s['profit_df'].copy()
                current_profit_df = current_profit_df.iloc[t:]
                current_profit_df = current_profit_df.reset_index(drop=True)
                current_profit_df = current_profit_df.iloc[:g]
                current_week_profit_list = current_profit_df['sum'].values
                
                
                
                #print(current_week_profit_list)
                
            
                for r in range(len(current_week_drawdown_list)):
                    
                    
                    
                    current_drawdown = current_week_drawdown_list[r] - current_week_profit_list[0]
                    
                    current_profit = current_week_profit_list[r] - current_week_profit_list[0]
                    
                    
                    if r == 0:
                        #skipping the first row
                        continue
                    
                    
                    if current_drawdown <= q:
                        
                        
                        current_week_calculated_profit = current_maximal_drawdown
                    
                        break
                        
                    
                    
                if current_week_calculated_profit == None:
                    
                    current_week_calculated_profit = current_profit
                    
                
                strategy_profit.append(current_week_calculated_profit)
                
            total_profit = sum(strategy_profit)
            
            profit_ratio = total_profit/(10*current_maximal_drawdown*-1)
            
            
            #print( 'start:', t+1, '  end:', g, '  drawdown:',q , '  profit:', total_profit, 'profit_ratio:',profit_ratio )
                
            start_date_index_list.append(t+1)

            cut_off_date_index_list.append(g)

            draw_down_list.append(q)

            total_spread_list.append(total_spread)

            profit_list.append(total_profit)

            profit_ratio_list.append(profit_ratio)
            
    print(t)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238


In [16]:
statistic_df['start_date_index'] = start_date_index_list


statistic_df['cut_off_date_index'] = cut_off_date_index_list

statistic_df['total_spread'] = total_spread_list

statistic_df['draw_down'] = draw_down_list





statistic_df['profit'] = profit_list

statistic_df['profit_ratio'] = profit_ratio_list





print(statistic_df)
statistic_df.to_csv('files/statistic_df.csv')

        start_date_index  cut_off_date_index  total_spread  draw_down  \
0                      1                   2          -200       -100   
1                      1                   2          -200       -200   
2                      1                   2          -200       -300   
3                      1                   2          -200       -400   
4                      1                   2          -200       -500   
...                  ...                 ...           ...        ...   
831715               239                   2          -200      -2500   
831716               239                   2          -200      -2600   
831717               239                   2          -200      -2700   
831718               239                   2          -200      -2800   
831719               239                   2          -200      -2900   

         profit  profit_ratio  
0      -46999.0    -15.666333  
1      -31107.0     -7.776750  
2      -22814.0     -4.5628